# **Building Q&A systems of SQL databases**
## **1. Introduction**

## **2. Setup**

### 2.1 Setting
 - Auto Reload
 - path for utils

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

### 2.2 Package Installation
- Install here ==> [package install](../../00_setup/setup.ipynb)

### 2.3. Bedrock Client 생성

In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [6]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


To use LangSmith (Not required)

In [7]:
os.environ["LANGCHAIN_API_KEY"] = <your api key>
os.environ["LANGCHAIN_TRACING_V2"] = "true"

아래 예는 Chinook 데이터베이스와의 SQLite 연결을 사용합니다. 다음 설치 단계에 따라 이 노트북과 같은 디렉터리에 Chinook.db를 만드세요 <BR>
`아래를 콘솔에서 실행합니다.` <BR><BR><BR>
*Run sqlite3 Chinook.db* <BR>
*Run .read Chinook_Sqlite.sql* <BR>
*Test SELECT * FROM Artist LIMIT 10;* <BR><BR>

이제 Chinhook.db가 디렉터리에 있으며 SQLAlchemy 기반 SQLDatabase 클래스를 사용하여 인터페이스할 수 있습니다:

In [8]:
from langchain_community.utilities import SQLDatabase

In [9]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Employee LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgar

잘됐네요! 쿼리할 수 있는 SQL 데이터베이스가 생겼습니다. 이제 이를 LLM에 연결해 보겠습니다.

## **3. Chain**

- 질문을 받아 SQL 쿼리로 바꾸고, 쿼리를 실행한 다음 그 결과를 사용하여 원래 질문에 답하는 간단한 체인을 만들어 보겠습니다.

In [10]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [11]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences": ["\n\nHuman:"]
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### 3.1 Convert question to SQL query (질문을 SQL 쿼리로 변환하기)
- SQL 체인 또는 에이전트의 첫 번째 단계는 사용자 입력을 받아 SQL 쿼리로 변환하는 것입니다.
- LangChain에는 이를 위한 기본 제공 체인이 있습니다: [create_sql_query_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sql_database.query.create_sql_query_chain.html)

In [32]:
from langchain.schema import StrOutputParser
from langchain.chains import create_sql_query_chain
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [33]:
chain = create_sql_query_chain(llm_text, db) | StrOutputParser()

In [34]:
query_generation_template='''

Human: You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date(\'now\') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Question: {input}

Skip the preamble and go straight into the sql.

Assistant: 
'''

chain.get_prompts()[0].template = query_generation_template
chain.get_prompts()[0].pretty_print()



Human: You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Question: {input}

Skip the preamble and go straight into the sql.

Assistant: 



In [35]:
response = chain.invoke(
    {"question": "직원들은 몇 명이 있나요?"}, ## How many employees are there
    #config={'callbacks': [ConsoleCallbackHandler()]}
)

 SELECT COUNT("EmployeeId") AS "Number of Employees" 
FROM "Employee"

In [36]:
db.run(response)

'[(8,)]'

In [37]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm_text, db)
write_query.get_prompts()[0].template = query_generation_template
chain = write_query | execute_query
chain.invoke(
    {"question": "How many employees are there",}
)

 SELECT COUNT(*) FROM "Employee"

'[(8,)]'

We can look at the LangSmith trace to get a better understanding of what this chain is doing. We can also inspect the chain directly for its prompts. Looking at the prompt (below), we can see that it is:

- Dialect-specific. In this case it references SQLite explicitly.
- Has definitions for all the available tables.
- Has three examples rows for each table.


이 기법은 예제 행을 표시하고 테이블에 대해 명시하면 성능이 향상된다는 [논문](https://arxiv.org/pdf/2204.00498.pdf)에서 영감을 얻은 것입니다. 다음과 같이 전체 프롬프트를 검사할 수도 있습니다:

### 3.2. Execute SQL query
- 이제 SQL 쿼리를 생성했으니 이를 실행해야 합니다.
- 이것은 SQL 체인을 만들 때 가장 위험한 부분입니다.
- 데이터에 대해 자동화된 쿼리를 실행해도 괜찮은지 신중하게 고려하세요.
- 데이터베이스 연결 권한을 최대한 최소화하세요.
- 쿼리 실행 전에 사람의 승인 단계를 체인에 추가하는 것을 고려하세요(아래 참조).

쿼리 실행을 체인에 쉽게 추가하기 위해 `QuerySQLDatabaseTool`을 사용할 수 있습니다:

In [41]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [42]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm_text, db)
write_query.get_prompts()[0].template = query_generation_template
chain = write_query | execute_query
chain.invoke(
    {"question": "How many employees are there",}
)

 SELECT COUNT(*) FROM "Employee"

'[(8,)]'

Answer the question
Now that we’ve got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer. We can do this by passing question and result to the LLM once more:

In [51]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

advanced_query_generation_template='''

Human: You are a expert.
Given the following user Question, corresponding SQLQuery, and SQLResult, Answer the user Question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: 


Assistant: 
'''

#chain.get_prompts()[0].template = advanced_query_generation_template
#chain.get_prompts()[0].pretty_print()

answer_prompt = PromptTemplate.from_template(advanced_query_generation_template)

answer = answer_prompt | llm_text | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke(
    {"question": "직원들은 몇 명이 있나요?"},
    #config={'callbacks': [ConsoleCallbackHandler()]}
)

 SELECT COUNT("EmployeeId") FROM "Employee" 직원들은 8명이 있습니다.

SQL 쿼리에서 Employee 테이블의 "EmployeeId" 컬럼의 행 개수를 COUNT 함수를 사용하여 계산하고 있습니다. SQL 결과에 따르면 직원 수는 8명입니다.

따라서 직원들은 8명이 있다고 답변할 수 있습니다.

' 직원들은 8명이 있습니다.\n\nSQL 쿼리에서 Employee 테이블의 "EmployeeId" 컬럼의 행 개수를 COUNT 함수를 사용하여 계산하고 있습니다. SQL 결과에 따르면 직원 수는 8명입니다.\n\n따라서 직원들은 8명이 있다고 답변할 수 있습니다.'

Next steps
For more complex query-generation, we may want to create few-shot prompts or add query-checking steps. For advanced techniques like this and more check out:

Prompting strategies: Advanced prompt engineering techniques.
Query checking: Add query validation and error handling.
Large databses: Techniques for working with large databases.

https://python.langchain.com/docs/use_cases/sql/quickstart